In [24]:
import numpy as np
import os

In [25]:
def extract_from_file(option):
    dir = os.getcwd()
    if option == "Train":
        file = dir+"/Toy Dataset/trainNN.txt"
    else:
        file = dir+"/Toy Dataset/testNN.txt"
    f = open(file,'r')
    num_array = np.array([[float(num) for num in line.split()] for line in f])
    return num_array

In [26]:
def get_feat_no_and_class_no(num_arr):
    feat_no = num_arr.shape[1] - 1  
    class_no = (int)(np.max(num_arr[:,-1]))
    
    return feat_no, class_no

In [27]:
def get_feat_no_and_classes(num_arr):
    feat_no = num_arr.shape[1] - 1  
    class_no = (int)(np.max(num_arr[:,-1]))
    
    return feat_no, class_no

In [28]:
def encode_class_values(Y):
    tmp = Y-1
    n_values = np.max(tmp.astype(int))+1
    encoded_class_values = np.eye(n_values)[tmp.astype(int)]
    return encoded_class_values

In [29]:
def activation_output(V):
        return 1.0 / (1.0 + np.exp(-V))

In [30]:
def activation_derivate(V):
    ac_output = activation_output(V)
    return ac_output * (1 - ac_output)

In [31]:
def get_X_and_Y(option):
    num_arr = extract_from_file(option)
    X, Y = num_arr[:,:-1],num_arr[:,-1]
    X_mod = (X - X.mean(axis = 0)) / X.std(axis = 0)

    if option == "Train":
        feat_no, class_no = get_feat_no_and_classes(num_arr)
        encoded_class_values = encode_class_values(Y) 
        return feat_no, class_no, X_mod, encoded_class_values

    return X_mod, Y

In [32]:
class Layer:
    def __init__(self, node_no, feat_no):
        
        np.random.seed(0)
        self.biases = np.random.randn(node_no, 1)
        self.pre_activation_output = None
        self.activation_output = None
        self.input = None
        np.random.seed(99)
        self.weights = np.random.randn(node_no, feat_no)

In [33]:
class NeuralNetwork:
    def __init__(self, feat_no, class_no, hidden_layer_sizes):
        self.feat_no = feat_no
        self.class_no = class_no
        self.hidden_layer_sizes = hidden_layer_sizes  
        self.layers = []  
        self.number_of_layers = None
        self.deltas = []  
        
    
    def add_layers(self):
        layer_sizes = [self.feat_no] + self.hidden_layer_sizes + [self.class_no]
        self.number_of_layers = len(layer_sizes) - 1;  
        for i in range(1, self.number_of_layers + 1):
            self.layers.append(Layer(layer_sizes[i], layer_sizes[i - 1]))


    def feedforward(self, X):
        prev_layer_output = X.T
        
        for layer in self.layers:
            layer.input = prev_layer_output
            layer.pre_activation_output = (layer.weights@ layer.input) + layer.biases
            layer.activation_output = activation_output(layer.pre_activation_output)
            prev_layer_output = layer.activation_output
        
        return prev_layer_output


    def compute_cost(self, train_Y):
        cost = np.sum((self.layers[-1].activation_output - train_Y.T) ** 2)
        return cost / 2.0


    def backward_propagation(self, train_Y):
        self.deltas = [0] * self.number_of_layers
        self.deltas[-1] = (self.layers[-1].activation_output - train_Y.T) * activation_derivate(self.layers[-1].pre_activation_output)
    

        for i in reversed(range(self.number_of_layers - 1)):
            self.deltas[i] = (self.layers[i + 1].weights.T@self.deltas[i + 1]) * activation_derivate(self.layers[i].pre_activation_output)
        #print(self.deltas)
    def update_weights(self, train_X, learning_rate):
        for i in range(self.number_of_layers):
            if i == 0:
                prev_layer_output = train_X.T
            else:
                prev_layer_output = self.layers[i - 1].activation_output
    
            delta_w = -learning_rate * (self.deltas[i]@ (prev_layer_output.T))
            self.layers[i].weights += delta_w
            
            delta_b = -learning_rate * np.sum(self.deltas[i], axis=1, keepdims=True)
            self.layers[i].biases += delta_b

    def train_neural_network(self, train_X, train_Y, learning_rate = 0.001, max_iterations = 10000):
        self.add_layers()
        previous_cost = -1
        for i in range(max_iterations):
            self.feedforward(train_X)
            current_cost = self.compute_cost(train_Y)
            
            if i == 0 or (current_cost < previous_cost and previous_cost - current_cost > 1e-20):
                # print(current_cost)
                previous_cost = current_cost
            else:
                print('iteration :' ,i+1)
                break
            
            self.backward_propagation(train_Y)
            self.update_weights(train_X, learning_rate)

            
    def predict_class(self, test_X):
        last_layer_output = self.feedforward(test_X)
        print(last_layer_output.shape)
        prediced_classes = []
        for row in last_layer_output.T:
            prediced_classes.append(np.argmax(row) + 1)

        return np.array(prediced_classes)

    def test_neural_network(self, test_X, test_Y):
        predicted_classes = self.predict_class(test_X)
        # matches = np.count_nonzero(predicted_classes == test_Y)
        # accuracy = matches / len(test_Y) * 100
        match_count = 0
        for i in range(len(test_Y)):
            if predicted_classes[i] == test_Y[i]:
                #print(test_X[i],test_Y[i],predicted_classes[i])
                match_count += 1
            else:
                print(i+1,test_X[i],test_Y[i],predicted_classes[i])
        
        accuracy = match_count / len(test_Y) * 100
        return accuracy


In [34]:
feat_no, class_no, train_X, train_Y = get_X_and_Y("Train")
test_X, test_Y = get_X_and_Y("Test")

nn = NeuralNetwork(feat_no, class_no, [3,3,3])
nn.train_neural_network(train_X, train_Y)
accuracy = nn.test_neural_network(test_X, test_Y)
print(accuracy)

(4, 500)
100.0


In [35]:
get_feat_no_and_class_no(extract_from_file("Train"))

(4, 4)